# Notebook 70: 微分の再発見 ― 変化率から勾配へ

## Derivative Intuition: From Rate of Change to Gradient

---

### このノートブックの位置づけ

**Unit 0.0「ニューラルエンジンの深部」** の第1章として、ニューラルネットワークの学習を支える最も基本的な数学的道具である **微分** を再発見します。

### 学習目標

1. **数値微分** を実装し、「微分とは何か」を体感する
2. **偏微分** と **勾配ベクトル** の概念を理解する
3. 勾配が **「最も急な上り坂の方向」** を指すことを可視化で確認する
4. ニューラルネットワークにおける微分の役割を直感的に理解する

### 前提知識

- 高校数学レベルの関数の概念
- Python / NumPy の基本操作

---

## 目次

1. [変化率とは何か：数値微分の実装](#1-変化率とは何か数値微分の実装)
2. [解析的微分 vs 数値微分](#2-解析的微分-vs-数値微分)
3. [多変数への拡張：偏微分](#3-多変数への拡張偏微分)
4. [勾配ベクトル：最急上昇方向](#4-勾配ベクトル最急上昇方向)
5. [勾配降下法への橋渡し](#5-勾配降下法への橋渡し)
6. [演習問題](#6-演習問題)
7. [まとめと次のステップ](#7-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定（環境に応じて調整）
plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# 再現性のためのシード
np.random.seed(42)

print("環境セットアップ完了")
print(f"NumPy version: {np.__version__}")

---

## 1. 変化率とは何か：数値微分の実装

### 1.1 微分の直感的な意味

微分とは、**「関数がある点でどれくらい急に変化しているか」** を測る操作です。

身近な例で考えてみましょう：

- **位置の微分 → 速度**：位置が時間に対してどれくらい変化しているか
- **速度の微分 → 加速度**：速度が時間に対してどれくらい変化しているか

数学的には、関数 $f(x)$ の点 $x$ における **微分係数（導関数の値）** は次のように定義されます：

$$
f'(x) = \lim_{h \to 0} \frac{f(x + h) - f(x)}{h}
$$

この式は「$x$ を微小量 $h$ だけ動かしたとき、$f(x)$ がどれだけ変化するか」の比率を表しています。

### 1.2 数値微分の実装

極限を直接計算することはできませんが、**十分に小さな $h$** を使えば近似できます。

#### 前進差分（Forward Difference）

$$
f'(x) \approx \frac{f(x + h) - f(x)}{h}
$$

#### 中心差分（Central Difference）― より精度が高い

$$
f'(x) \approx \frac{f(x + h) - f(x - h)}{2h}
$$

In [ ]:
def numerical_diff_forward(f, x, h=1e-5):
    """前進差分による数値微分"""
    return (f(x + h) - f(x)) / h


def numerical_diff_central(f, x, h=1e-5):
    """中心差分による数値微分（より高精度）"""
    return (f(x + h) - f(x - h)) / (2 * h)


# テスト関数: f(x) = x^2
# 解析的な導関数: f'(x) = 2x
def f(x):
    return x ** 2


def f_prime_analytical(x):
    """解析的な導関数"""
    return 2 * x


# x = 3 での微分値を計算
x = 3.0
analytical = f_prime_analytical(x)
forward = numerical_diff_forward(f, x)
central = numerical_diff_central(f, x)

print(f"f(x) = x² の x = {x} における微分値")
print(f"="*40)
print(f"解析的微分:     {analytical:.10f}")
print(f"前進差分:       {forward:.10f}  (誤差: {abs(analytical - forward):.2e})")
print(f"中心差分:       {central:.10f}  (誤差: {abs(analytical - central):.2e})")

### 1.3 なぜ中心差分の方が精度が高いのか？

テイラー展開を使うと、誤差の次数を分析できます：

- **前進差分**: $O(h)$ の誤差（$h$ に比例）
- **中心差分**: $O(h^2)$ の誤差（$h^2$ に比例、より小さい）

$h$ を小さくしたときの挙動を確認してみましょう。

In [ ]:
# h を変化させたときの誤差を比較
h_values = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-10, 1e-12]
x = 3.0
analytical = f_prime_analytical(x)

forward_errors = []
central_errors = []

for h in h_values:
    forward_errors.append(abs(numerical_diff_forward(f, x, h) - analytical))
    central_errors.append(abs(numerical_diff_central(f, x, h) - analytical))

# 可視化
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(h_values, forward_errors, 'o-', label='前進差分', linewidth=2, markersize=8)
ax.loglog(h_values, central_errors, 's-', label='中心差分', linewidth=2, markersize=8)

# 理論的な誤差オーダーの参照線
ax.loglog(h_values[:6], [h * 10 for h in h_values[:6]], '--', alpha=0.5, label='O(h) 参照')
ax.loglog(h_values[:6], [h**2 * 100 for h in h_values[:6]], '--', alpha=0.5, label='O(h²) 参照')

ax.set_xlabel('h（刻み幅）', fontsize=12)
ax.set_ylabel('絶対誤差', fontsize=12)
ax.set_title('数値微分の精度：h を小さくすると何が起きるか？', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(1e-13, 1)
ax.set_ylim(1e-12, 1)

plt.tight_layout()
plt.show()

print("\n【重要な観察】")
print("1. 中心差分は前進差分より一貫して精度が高い")
print("2. h が小さすぎると（< 1e-8）、浮動小数点の丸め誤差で精度が悪化する")
print("3. 実用的には h = 1e-5 ~ 1e-7 程度が良いバランス")

### 1.4 接線の可視化：微分の幾何学的意味

微分係数 $f'(a)$ は、点 $(a, f(a))$ における接線の **傾き** です。

接線の方程式：
$$
y = f'(a)(x - a) + f(a)
$$

In [ ]:
def plot_tangent_line(f, f_prime, a, x_range=(-3, 3), title="関数と接線"):
    """関数とその接線を可視化する"""
    x = np.linspace(x_range[0], x_range[1], 200)
    y = f(x)
    
    # 接線の計算
    slope = f_prime(a)
    tangent = slope * (x - a) + f(a)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # 関数のプロット
    ax.plot(x, y, 'b-', linewidth=2.5, label=f'$f(x)$')
    
    # 接線のプロット
    ax.plot(x, tangent, 'r--', linewidth=2, label=f'接線（傾き = {slope:.2f}）')
    
    # 接点のマーク
    ax.plot(a, f(a), 'go', markersize=12, label=f'接点 ({a}, {f(a):.2f})', zorder=5)
    
    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel('y', fontsize=12)
    ax.set_title(title, fontsize=14)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(min(y) - 1, max(y) + 1)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    
    plt.tight_layout()
    return fig, ax


# f(x) = x^2 の接線を複数点で描画
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, a in enumerate([-2, 0, 1.5]):
    x = np.linspace(-3, 3, 200)
    y = x ** 2
    slope = 2 * a  # f'(x) = 2x
    tangent = slope * (x - a) + a**2
    
    axes[idx].plot(x, y, 'b-', linewidth=2.5, label='$f(x) = x^2$')
    axes[idx].plot(x, tangent, 'r--', linewidth=2, label=f'接線 (傾き={slope:.1f})')
    axes[idx].plot(a, a**2, 'go', markersize=12, zorder=5)
    axes[idx].set_xlabel('x')
    axes[idx].set_ylabel('y')
    axes[idx].set_title(f'x = {a} での接線', fontsize=12)
    axes[idx].legend(fontsize=9)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim(-2, 10)
    axes[idx].set_xlim(-3, 3)

plt.suptitle('$f(x) = x^2$ の各点における接線', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("【観察ポイント】")
print("・x = -2: 傾き -4（左下がり）→ x を増やすと f(x) は減少")
print("・x = 0:  傾き 0（水平）    → 極小点")
print("・x = 1.5: 傾き 3（右上がり）→ x を増やすと f(x) は増加")

---

## 2. 解析的微分 vs 数値微分

### 2.1 解析的微分

数学的な公式を使って導関数を求める方法です。

**基本的な微分公式：**

| 関数 $f(x)$ | 導関数 $f'(x)$ |
|-------------|----------------|
| $x^n$ | $nx^{n-1}$ |
| $e^x$ | $e^x$ |
| $\ln(x)$ | $1/x$ |
| $\sin(x)$ | $\cos(x)$ |
| $\cos(x)$ | $-\sin(x)$ |

### 2.2 ニューラルネットワークでよく使う関数の微分

In [ ]:
# シグモイド関数とその導関数
def sigmoid(x):
    """シグモイド関数: σ(x) = 1 / (1 + e^(-x))"""
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    """シグモイドの導関数: σ'(x) = σ(x)(1 - σ(x))"""
    s = sigmoid(x)
    return s * (1 - s)


# ReLU関数とその導関数
def relu(x):
    """ReLU関数: max(0, x)"""
    return np.maximum(0, x)


def relu_derivative(x):
    """ReLUの導関数: 1 if x > 0 else 0"""
    return (x > 0).astype(float)


# tanh関数とその導関数
def tanh(x):
    """tanh関数"""
    return np.tanh(x)


def tanh_derivative(x):
    """tanhの導関数: 1 - tanh²(x)"""
    return 1 - np.tanh(x)**2


# 可視化
x = np.linspace(-5, 5, 500)

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# シグモイド
axes[0, 0].plot(x, sigmoid(x), 'b-', linewidth=2.5)
axes[0, 0].set_title('Sigmoid: $\\sigma(x) = \\frac{1}{1+e^{-x}}$', fontsize=12)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_ylim(-0.1, 1.1)

axes[1, 0].plot(x, sigmoid_derivative(x), 'r-', linewidth=2.5)
axes[1, 0].set_title("$\\sigma'(x) = \\sigma(x)(1-\\sigma(x))$", fontsize=12)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0.25, color='gray', linestyle='--', alpha=0.5)
axes[1, 0].text(3, 0.26, '最大値 = 0.25', fontsize=10)

# ReLU
axes[0, 1].plot(x, relu(x), 'b-', linewidth=2.5)
axes[0, 1].set_title('ReLU: $\\max(0, x)$', fontsize=12)
axes[0, 1].grid(True, alpha=0.3)

axes[1, 1].plot(x, relu_derivative(x), 'r-', linewidth=2.5)
axes[1, 1].set_title("ReLU': 1 (x>0), 0 (x≤0)", fontsize=12)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim(-0.1, 1.5)

# tanh
axes[0, 2].plot(x, tanh(x), 'b-', linewidth=2.5)
axes[0, 2].set_title('tanh: $\\frac{e^x - e^{-x}}{e^x + e^{-x}}$', fontsize=12)
axes[0, 2].grid(True, alpha=0.3)
axes[0, 2].set_ylim(-1.2, 1.2)

axes[1, 2].plot(x, tanh_derivative(x), 'r-', linewidth=2.5)
axes[1, 2].set_title("$\\tanh'(x) = 1 - \\tanh^2(x)$", fontsize=12)
axes[1, 2].grid(True, alpha=0.3)

for ax in axes.flat:
    ax.set_xlabel('x')
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)

axes[0, 0].set_ylabel('f(x)')
axes[1, 0].set_ylabel("f'(x)")

plt.suptitle('活性化関数とその導関数', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("【重要な観察】")
print("・Sigmoid の導関数は最大でも 0.25 → 深い層で勾配が消失しやすい")
print("・ReLU の導関数は 0 か 1 → 勾配がそのまま伝播（ただし負の入力では死ぬ）")
print("・tanh の導関数は最大 1 → Sigmoid より勾配消失しにくい")

### 2.3 数値微分で検証する

解析的に求めた導関数が正しいか、数値微分で検証してみましょう。これは **勾配チェック（Gradient Checking）** と呼ばれ、ニューラルネットワークの実装で非常に重要な検証手法です。

In [ ]:
def gradient_check(f, f_prime, test_points, h=1e-5, tol=1e-6):
    """解析的な導関数を数値微分で検証する"""
    print(f"{'x':>8} | {'解析的':>12} | {'数値微分':>12} | {'相対誤差':>12} | 結果")
    print("-" * 65)
    
    all_passed = True
    for x in test_points:
        analytical = f_prime(x)
        numerical = numerical_diff_central(f, x, h)
        
        # 相対誤差の計算（ゼロ除算を避ける）
        if abs(analytical) > 1e-10:
            relative_error = abs(analytical - numerical) / abs(analytical)
        else:
            relative_error = abs(analytical - numerical)
        
        passed = relative_error < tol
        all_passed = all_passed and passed
        status = "✓ OK" if passed else "✗ NG"
        
        print(f"{x:>8.2f} | {analytical:>12.6f} | {numerical:>12.6f} | {relative_error:>12.2e} | {status}")
    
    return all_passed


# シグモイド関数の勾配チェック
print("【Sigmoid関数の勾配チェック】\n")
test_points = np.array([-2.0, -1.0, 0.0, 1.0, 2.0])
gradient_check(sigmoid, sigmoid_derivative, test_points)

print("\n" + "="*65 + "\n")

# tanh関数の勾配チェック
print("【tanh関数の勾配チェック】\n")
gradient_check(tanh, tanh_derivative, test_points)

---

## 3. 多変数への拡張：偏微分

### 3.1 多変数関数とは

ニューラルネットワークは **多数のパラメータ（重み）** を持ちます。損失関数はこれらすべてのパラメータの関数です：

$$
L = L(w_1, w_2, \ldots, w_n)
$$

このような多変数関数を微分するには、**偏微分** を使います。

### 3.2 偏微分の定義

関数 $f(x, y)$ の $x$ に関する偏微分は、**$y$ を固定して** $x$ だけを変化させたときの変化率です：

$$
\frac{\partial f}{\partial x} = \lim_{h \to 0} \frac{f(x + h, y) - f(x, y)}{h}
$$

同様に、$y$ に関する偏微分は：

$$
\frac{\partial f}{\partial y} = \lim_{h \to 0} \frac{f(x, y + h) - f(x, y)}{h}
$$

In [ ]:
# 2変数関数の例: f(x, y) = x² + y²
def f_2d(x, y):
    return x**2 + y**2


# 解析的な偏微分
def df_dx(x, y):
    """∂f/∂x = 2x"""
    return 2 * x


def df_dy(x, y):
    """∂f/∂y = 2y"""
    return 2 * y


# 数値偏微分
def numerical_partial_x(f, x, y, h=1e-5):
    """xに関する数値偏微分"""
    return (f(x + h, y) - f(x - h, y)) / (2 * h)


def numerical_partial_y(f, x, y, h=1e-5):
    """yに関する数値偏微分"""
    return (f(x, y + h) - f(x, y - h)) / (2 * h)


# テスト
test_point = (3.0, 4.0)
x, y = test_point

print(f"f(x, y) = x² + y² の点 ({x}, {y}) における偏微分")
print("="*50)
print(f"\n∂f/∂x:")
print(f"  解析的: {df_dx(x, y):.6f}")
print(f"  数値:   {numerical_partial_x(f_2d, x, y):.6f}")
print(f"\n∂f/∂y:")
print(f"  解析的: {df_dy(x, y):.6f}")
print(f"  数値:   {numerical_partial_y(f_2d, x, y):.6f}")

### 3.3 等高線図で偏微分を可視化

等高線図は、3次元の関数を2次元で表現する方法です。偏微分の幾何学的意味を理解しましょう。

In [ ]:
# 等高線図と偏微分の方向を可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# メッシュグリッドの作成
x_range = np.linspace(-4, 4, 100)
y_range = np.linspace(-4, 4, 100)
X, Y = np.meshgrid(x_range, y_range)
Z = f_2d(X, Y)

# 左: 等高線図
contour = axes[0].contour(X, Y, Z, levels=15, cmap='viridis')
axes[0].clabel(contour, inline=True, fontsize=8)
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('$f(x,y) = x^2 + y^2$ の等高線図', fontsize=12)
axes[0].set_aspect('equal')
axes[0].grid(True, alpha=0.3)

# 特定の点での偏微分を矢印で表示
point = (2.0, 1.5)
axes[0].plot(*point, 'ro', markersize=12, label=f'点 {point}')

# ∂f/∂x の方向（x方向のみの変化）
scale = 0.3
axes[0].annotate('', xy=(point[0] + scale * df_dx(*point), point[1]),
                 xytext=point, arrowprops=dict(arrowstyle='->', color='red', lw=2))
axes[0].text(point[0] + 0.3, point[1] - 0.5, f'∂f/∂x = {df_dx(*point):.1f}', 
             color='red', fontsize=11)

# ∂f/∂y の方向（y方向のみの変化）
axes[0].annotate('', xy=(point[0], point[1] + scale * df_dy(*point)),
                 xytext=point, arrowprops=dict(arrowstyle='->', color='blue', lw=2))
axes[0].text(point[0] - 1.5, point[1] + 0.8, f'∂f/∂y = {df_dy(*point):.1f}', 
             color='blue', fontsize=11)

axes[0].legend(loc='upper left')

# 右: xを固定してyを変化させたときのスライス
x_fixed = 2.0
y_vals = np.linspace(-4, 4, 100)
z_slice = f_2d(x_fixed, y_vals)

axes[1].plot(y_vals, z_slice, 'b-', linewidth=2.5, label=f'$f({x_fixed}, y) = {x_fixed}^2 + y^2$')

# 接線を描画
y_point = 1.5
slope = df_dy(x_fixed, y_point)
tangent = slope * (y_vals - y_point) + f_2d(x_fixed, y_point)
axes[1].plot(y_vals, tangent, 'r--', linewidth=2, label=f'接線（傾き = {slope:.1f}）')
axes[1].plot(y_point, f_2d(x_fixed, y_point), 'go', markersize=12)

axes[1].set_xlabel('y')
axes[1].set_ylabel('f')
axes[1].set_title(f'x = {x_fixed} で固定したときのスライス', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(-2, 25)

plt.tight_layout()
plt.show()

print("【偏微分の意味】")
print("・∂f/∂x: 『y を固定して x だけ動かしたとき、f がどれだけ変化するか』")
print("・∂f/∂y: 『x を固定して y だけ動かしたとき、f がどれだけ変化するか』")

---

## 4. 勾配ベクトル：最急上昇方向

### 4.1 勾配の定義

偏微分を並べてベクトルにしたものを **勾配（Gradient）** と呼びます：

$$
\nabla f = \left( \frac{\partial f}{\partial x}, \frac{\partial f}{\partial y} \right)
$$

### 4.2 勾配の重要な性質

**勾配ベクトルは、関数が最も急に増加する方向を指す**

これがニューラルネットワークの学習で本質的に重要な性質です：

- 損失関数 $L$ を **減少** させたい
- 勾配 $\nabla L$ は $L$ が **増加** する方向を指す
- したがって、**勾配の逆方向（$-\nabla L$）** に進めば $L$ は減少する

In [ ]:
def compute_gradient(f, x, y, h=1e-5):
    """数値微分で勾配を計算"""
    grad_x = (f(x + h, y) - f(x - h, y)) / (2 * h)
    grad_y = (f(x, y + h) - f(x, y - h)) / (2 * h)
    return np.array([grad_x, grad_y])


# 勾配場（Gradient Field）の可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 左: 勾配ベクトル場
x_range = np.linspace(-3, 3, 15)
y_range = np.linspace(-3, 3, 15)
X_grid, Y_grid = np.meshgrid(x_range, y_range)

# 各点での勾配を計算
U = 2 * X_grid  # ∂f/∂x = 2x
V = 2 * Y_grid  # ∂f/∂y = 2y

# 等高線
X_fine, Y_fine = np.meshgrid(np.linspace(-3, 3, 100), np.linspace(-3, 3, 100))
Z_fine = f_2d(X_fine, Y_fine)
contour = axes[0].contour(X_fine, Y_fine, Z_fine, levels=15, cmap='coolwarm', alpha=0.6)

# 勾配ベクトル場（矢印）
axes[0].quiver(X_grid, Y_grid, U, V, color='blue', alpha=0.7, scale=50)

axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('勾配ベクトル場：矢印は f が最も急に増加する方向', fontsize=12)
axes[0].set_aspect('equal')
axes[0].grid(True, alpha=0.3)

# 右: 負の勾配（勾配降下の方向）
contour2 = axes[1].contour(X_fine, Y_fine, Z_fine, levels=15, cmap='coolwarm', alpha=0.6)

# 負の勾配ベクトル場
axes[1].quiver(X_grid, Y_grid, -U, -V, color='green', alpha=0.7, scale=50)

# 最小点をマーク
axes[1].plot(0, 0, 'r*', markersize=20, label='最小点 (0, 0)')

axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_title('負の勾配：f を減少させる方向（最小点へ向かう）', fontsize=12)
axes[1].set_aspect('equal')
axes[1].grid(True, alpha=0.3)
axes[1].legend(loc='upper right')

plt.tight_layout()
plt.show()

print("【核心的な洞察】")
print("・勾配ベクトルはすべて外向き（原点から離れる方向）= f が増加する方向")
print("・負の勾配はすべて内向き（原点に向かう方向）= f が減少する方向")
print("・これが『勾配降下法』の基本原理")

### 4.3 3D曲面上での勾配の意味

勾配の大きさは「傾斜の急さ」を表し、方向は「最も急な上り坂」を示します。

In [ ]:
# 3D曲面と勾配の可視化
fig = plt.figure(figsize=(14, 5))

# 3D曲面
ax1 = fig.add_subplot(1, 2, 1, projection='3d')

X = np.linspace(-3, 3, 50)
Y = np.linspace(-3, 3, 50)
X, Y = np.meshgrid(X, Y)
Z = f_2d(X, Y)

surf = ax1.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8, edgecolor='none')

# 特定の点とその勾配方向を表示
point = (2.0, 1.0)
z_point = f_2d(*point)
ax1.scatter(*point, z_point, color='red', s=100, label=f'点 {point}')

# 勾配の方向を曲面上に投影
grad = np.array([2 * point[0], 2 * point[1]])  # 解析的な勾配
grad_norm = np.linalg.norm(grad)
grad_unit = grad / grad_norm * 0.5  # 単位ベクトルにスケーリング

ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('f(x,y)')
ax1.set_title('$f(x,y) = x^2 + y^2$ の3D曲面', fontsize=12)
ax1.view_init(elev=30, azim=45)

# 2D等高線図と勾配
ax2 = fig.add_subplot(1, 2, 2)

X_fine = np.linspace(-3, 3, 100)
Y_fine = np.linspace(-3, 3, 100)
X_fine, Y_fine = np.meshgrid(X_fine, Y_fine)
Z_fine = f_2d(X_fine, Y_fine)

contour = ax2.contourf(X_fine, Y_fine, Z_fine, levels=20, cmap='viridis', alpha=0.8)
plt.colorbar(contour, ax=ax2, label='f(x,y)')

# 勾配ベクトルを描画
ax2.arrow(point[0], point[1], grad_unit[0], grad_unit[1],
          head_width=0.15, head_length=0.1, fc='red', ec='red', linewidth=2,
          label=f'勾配 ∇f = ({grad[0]:.1f}, {grad[1]:.1f})')

# 負の勾配（降下方向）
ax2.arrow(point[0], point[1], -grad_unit[0], -grad_unit[1],
          head_width=0.15, head_length=0.1, fc='lime', ec='lime', linewidth=2,
          label='降下方向 -∇f')

ax2.plot(*point, 'ko', markersize=10)
ax2.plot(0, 0, 'w*', markersize=15, markeredgecolor='black')
ax2.text(0.1, 0.3, '最小点', fontsize=10, color='white')

ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('等高線図と勾配ベクトル', fontsize=12)
ax2.set_aspect('equal')
ax2.legend(loc='upper left', fontsize=9)

plt.tight_layout()
plt.show()

print(f"点 {point} での勾配: ∇f = ({grad[0]:.1f}, {grad[1]:.1f})")
print(f"勾配の大きさ |∇f| = {grad_norm:.2f}（傾斜の急さ）")

---

## 5. 勾配降下法への橋渡し

### 5.1 最適化問題としてのニューラルネットワーク学習

ニューラルネットワークの学習は、次の最適化問題を解くことに帰着します：

$$
\min_{\mathbf{w}} L(\mathbf{w})
$$

ここで：
- $\mathbf{w}$：ネットワークの全パラメータ（重みとバイアス）
- $L$：損失関数（予測と正解のズレを測る）

### 5.2 勾配降下法のアルゴリズム

1. 現在のパラメータ $\mathbf{w}$ で勾配 $\nabla L(\mathbf{w})$ を計算
2. 勾配の逆方向に少しだけ移動：
   $$\mathbf{w}_{\text{new}} = \mathbf{w} - \eta \nabla L(\mathbf{w})$$
3. 収束するまで繰り返す

$\eta$ は **学習率（Learning Rate）** と呼ばれるハイパーパラメータです。

In [ ]:
def gradient_descent_2d(f, grad_f, start, learning_rate=0.1, n_iterations=50):
    """2変数関数の勾配降下法"""
    path = [np.array(start)]
    current = np.array(start, dtype=float)
    
    for i in range(n_iterations):
        grad = grad_f(current[0], current[1])
        current = current - learning_rate * grad
        path.append(current.copy())
    
    return np.array(path)


def analytical_gradient(x, y):
    """f(x,y) = x² + y² の勾配"""
    return np.array([2*x, 2*y])


# 異なる学習率での軌跡を比較
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

learning_rates = [0.01, 0.1, 0.5]
start_point = (2.5, 2.0)

# 等高線の準備
X = np.linspace(-3, 3, 100)
Y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(X, Y)
Z = f_2d(X, Y)

for ax, lr in zip(axes, learning_rates):
    # 等高線
    contour = ax.contour(X, Y, Z, levels=15, cmap='viridis', alpha=0.6)
    
    # 勾配降下の軌跡
    path = gradient_descent_2d(f_2d, analytical_gradient, start_point, 
                               learning_rate=lr, n_iterations=30)
    
    # 軌跡をプロット
    ax.plot(path[:, 0], path[:, 1], 'ro-', markersize=4, linewidth=1.5, alpha=0.8)
    ax.plot(path[0, 0], path[0, 1], 'g^', markersize=12, label='開始点')
    ax.plot(path[-1, 0], path[-1, 1], 'bs', markersize=10, label='終了点')
    ax.plot(0, 0, 'k*', markersize=15, label='最小点')
    
    # 各ステップでの損失値を計算
    final_loss = f_2d(path[-1, 0], path[-1, 1])
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f'学習率 η = {lr}\n(最終損失: {final_loss:.4f})', fontsize=12)
    ax.set_aspect('equal')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)

plt.suptitle('勾配降下法：学習率による収束速度の違い', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("【学習率の影響】")
print("・η = 0.01: ゆっくり収束（安定だが遅い）")
print("・η = 0.1:  バランスが良い")
print("・η = 0.5:  速いが、大きすぎると発散の危険性")

In [ ]:
# 学習曲線（損失の推移）を可視化
fig, ax = plt.subplots(figsize=(10, 6))

for lr in [0.01, 0.05, 0.1, 0.3, 0.5]:
    path = gradient_descent_2d(f_2d, analytical_gradient, start_point,
                               learning_rate=lr, n_iterations=50)
    losses = [f_2d(p[0], p[1]) for p in path]
    ax.plot(losses, label=f'η = {lr}', linewidth=2)

ax.set_xlabel('イテレーション', fontsize=12)
ax.set_ylabel('損失 L(w)', fontsize=12)
ax.set_title('学習曲線：損失関数の減少', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')

plt.tight_layout()
plt.show()

print("【対数スケールで見る収束】")
print("・適切な学習率では、損失が指数関数的に減少")
print("・学習率が大きいほど最初の減少は速いが、振動のリスクも")

---

## 6. 演習問題

### 演習 6.1: 数値微分の実装確認

以下の関数について、数値微分と解析的微分を比較してください。

$$
f(x) = \sin(x) \cdot e^{-x^2/2}
$$

In [ ]:
# 演習 6.1: 解答欄

def exercise_f(x):
    """f(x) = sin(x) * exp(-x²/2)"""
    # TODO: 実装してください
    pass


def exercise_f_prime(x):
    """f'(x) の解析的な導関数（積の微分法則を使用）"""
    # ヒント: (fg)' = f'g + fg'
    # f = sin(x), f' = cos(x)
    # g = exp(-x²/2), g' = -x * exp(-x²/2)
    # TODO: 実装してください
    pass


# テストコード（実装後にコメントを外して実行）
# test_points = np.array([-2.0, -1.0, 0.0, 0.5, 1.0, 2.0])
# gradient_check(exercise_f, exercise_f_prime, test_points)

### 演習 6.2: 2変数関数の勾配降下

以下の **ローゼンブロック関数**（最適化のベンチマークとして有名）に対して勾配降下法を適用してください。

$$
f(x, y) = (1 - x)^2 + 100(y - x^2)^2
$$

最小値は $(x, y) = (1, 1)$ で $f = 0$ です。

In [ ]:
# 演習 6.2: 解答欄

def rosenbrock(x, y):
    """ローゼンブロック関数"""
    # TODO: 実装してください
    pass


def rosenbrock_gradient(x, y):
    """ローゼンブロック関数の勾配"""
    # ∂f/∂x = -2(1-x) - 400x(y - x²)
    # ∂f/∂y = 200(y - x²)
    # TODO: 実装してください
    pass


# 勾配降下法を試す
# start_point = (-1.0, 1.0)
# path = gradient_descent_2d(rosenbrock, rosenbrock_gradient, start_point,
#                            learning_rate=0.001, n_iterations=5000)

# 可視化コード（実装後にコメントを外して実行）
# X = np.linspace(-2, 2, 100)
# Y = np.linspace(-1, 3, 100)
# X, Y = np.meshgrid(X, Y)
# Z = rosenbrock(X, Y)
# plt.contour(X, Y, np.log(Z + 1), levels=30)
# plt.plot(path[:, 0], path[:, 1], 'r.-', alpha=0.5)
# plt.show()

### 演習 6.3: シグモイドの飽和問題

シグモイド関数の導関数が、入力 $x$ が大きい（または小さい）ときにほぼ 0 になることを確認してください。これが「勾配消失」の原因の一つです。

In [ ]:
# 演習 6.3: 解答欄

# シグモイドの導関数値を様々な入力で計算
x_values = [-10, -5, -2, -1, 0, 1, 2, 5, 10]

print("x の値とシグモイドの導関数 σ'(x) = σ(x)(1-σ(x))")
print("="*50)

# TODO: 各 x_values に対して sigmoid_derivative を計算し、
# 値が極端に小さくなる様子を確認してください

# for x in x_values:
#     deriv = sigmoid_derivative(x)
#     print(f"x = {x:>4}: σ'(x) = {deriv:.10f}")

---

## 7. まとめと次のステップ

### このノートブックで学んだこと

1. **数値微分**：$f'(x) \approx \frac{f(x+h) - f(x-h)}{2h}$ で任意の関数の微分を近似できる

2. **偏微分**：多変数関数の各変数に対する変化率

3. **勾配ベクトル**：偏微分を並べたベクトル $\nabla f = (\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y})$

4. **勾配の方向**：関数が最も急に増加する方向を指す

5. **勾配降下法**：$\mathbf{w}_{\text{new}} = \mathbf{w} - \eta \nabla L$ で損失を最小化

### ニューラルネットワークとの接続

| 概念 | ニューラルネットワークでの役割 |
|------|-------------------------------|
| 関数 $f$ | モデルの出力（予測値） |
| 変数 $x, y, \ldots$ | 重み $w_1, w_2, \ldots$ とバイアス |
| 損失関数 $L$ | 予測と正解のズレ（MSE, Cross Entropyなど） |
| 勾配 $\nabla L$ | 各パラメータの「責任度」 |
| 勾配降下 | パラメータの更新（学習） |

### 次のノートブック（71: 連鎖律の解剖）への橋渡し

実際のニューラルネットワークでは、関数が **何層にも重なった合成関数** になっています：

$$
y = f_3(f_2(f_1(x)))
$$

このような合成関数の微分には **連鎖律（Chain Rule）** が必要です。次のノートブックでは：

- 連鎖律の数学的導出
- 複雑な合成関数の手計算練習
- 計算グラフ表現への準備

を学びます。

---

## 参考文献

1. Goodfellow, I., Bengio, Y., & Courville, A. (2016). *Deep Learning*. MIT Press. Chapter 4: Numerical Computation
2. Bishop, C. M. (2006). *Pattern Recognition and Machine Learning*. Springer. Appendix D: Calculus of Variations
3. 斎藤康毅 (2016). 『ゼロから作るDeep Learning』. O'Reilly Japan. 第4章: ニューラルネットワークの学習